# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

burdan sonraki ksıım ucak rezervasyonu için ekeldim

In [60]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        city = arguments.get("destination_city")
        price = get_ticket_price(city)
        content = json.dumps({"destination_city": city, "price": price})

    elif tool_name == "make_reservation":
        from_city = arguments.get("departure_city")
        to_city = arguments.get("destination_city")
        result = make_reservation(from_city, to_city)
        content = json.dumps({"message": result})

    else:
        content = json.dumps({"message": "Unknown tool"})

    response = {
        "role": "tool",
        "content": content,
        "tool_call_id": tool_call.id
    }

    return response, arguments.get("destination_city")



In [61]:
tools = [get_ticket_price_tool, make_reservation_tool]

In [62]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


In [1]:
# 1. Gerekli kütüphaneleri yükle
import os
import json
import random
import string
from datetime import datetime, timedelta
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
# 2. Ortam değişkenlerini yükle
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()
MODEL = "gpt-4o-mini"


In [3]:
# 3. Uçak bileti fiyatları sözlüğü (çift şehirli anahtarlarla)
ticket_prices = {
    ("istanbul", "berlin"): "$499",
    ("istanbul", "tokyo"): "$1400",
    ("ankara", "berlin"): "$550",
    ("izmir", "tokyo"): "$1350",
    ("antalya", "paris"): "$790",
    ("istanbul", "paris"): "$899"
}


In [4]:
# 4. Bilet fiyatı fonksiyonu (güvenli)
def get_ticket_price(departure_city, destination_city):
    if not departure_city or not destination_city:
        return "Error: both departure and destination cities are required."
    key = (departure_city.lower(), destination_city.lower())
    price = ticket_prices.get(key)
    if price:
        return price
    else:
        return f"Estimated price from {departure_city.title()} to {destination_city.title()} is around $1000."


In [5]:
# 5. PNR üretici & rezervasyon fonksiyonu
def generate_pnr():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

def make_reservation(departure_city, destination_city):
    pnr = generate_pnr()
    flight_time = datetime.now() + timedelta(days=1)
    time_str = flight_time.strftime("%d %B %Y at %H:%M")
    return (
        f"Your reservation from {departure_city.title()} to {destination_city.title()} is confirmed.\n"
        f"Flight departs on {time_str}.\n"
        f"Reservation Code (PNR): {pnr}.\n"
        f"Seat selection will be available during check-in."
    )


In [6]:

# 6. Tool tanımları
get_ticket_price_tool = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Returns the ticket price between a departure and destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {
                    "type": "string",
                    "description": "City where the passenger will depart from"
                },
                "destination_city": {
                    "type": "string",
                    "description": "City where the passenger wants to travel"
                }
            },
            "required": ["departure_city", "destination_city"]
        }
    }
}

make_reservation_tool = {
    "type": "function",
    "function": {
        "name": "make_reservation",
        "description": "Makes a reservation from a departure to a destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {
                    "type": "string",
                    "description": "City where the passenger will depart from"
                },
                "destination_city": {
                    "type": "string",
                    "description": "City where the passenger wants to travel"
                }
            },
            "required": ["departure_city", "destination_city"]
        }
    }
}


In [7]:
# 7. Tools listesi
tools = [get_ticket_price_tool, make_reservation_tool]

In [8]:
# 8. Sistem mesajı
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Never call any tools unless both the departure and destination cities are clearly provided. "
    "If the user says only a destination, such as 'I want to go to Berlin', ask them: "
    "'From which city would you like to fly to Berlin?' "
)

In [16]:
# 9. Tool çağrısı yöneticisi
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        from_city = arguments.get("departure_city")
        to_city = arguments.get("destination_city")
        price = get_ticket_price(from_city, to_city)
        content = json.dumps({
            "departure_city": from_city,
            "destination_city": to_city,
            "price": price
        })

    elif tool_name == "make_reservation":
        from_city = arguments.get("departure_city")
        to_city = arguments.get("destination_city")
        result = make_reservation(from_city, to_city)
        content = json.dumps({"message": result})

    else:
        content = json.dumps({"message": "Unknown tool called."})

    response = {
        "role": "tool",
        "content": content,
        "tool_call_id": tool_call.id
    }

    return response, arguments.get("destination_city")

In [17]:

# 10. Ana sohbet fonksiyonu
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # 1. İlk LLM çağrısı (tool_call yapılabilir)
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # 2. Eğer LLM bir tool_call yaptıysa:
    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message              # tool_call içeren assistant mesajı
        tool_response, _ = handle_tool_call(assistant_message)       # Python'da tool çalıştır, yanıtını al

        # ❗Tool çağrısına karşılık gelen cevabı ekliyoruz
        messages.append(assistant_message)   # role: "assistant", tool_calls içeriyor
        messages.append(tool_response)       # role: "tool", tool_call_id eşleşiyor

        # 3. LLM'e tekrar gönder → artık elinde hem tool_call hem tool cevabı var
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    # 4. Son cevabı döndür
    return response.choices[0].message.content



In [23]:
# ✈️ FlightAI Chatbot - Jupyter için Baştan Sona Kod

# 1. Gerekli kütüphaneleri yükle
import os
import json
import random
import string
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# 2. Ortam değişkenlerini yükle
load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-4o-mini"

# 3. Uçak bileti fiyatları sözlüğü (çift şehirli anahtarlarla)
ticket_prices = {
    ("istanbul", "berlin"): "$499",
    ("istanbul", "tokyo"): "$1400",
    ("ankara", "berlin"): "$550",
    ("izmir", "tokyo"): "$1350",
    ("antalya", "paris"): "$790",
    ("istanbul", "paris"): "$899"
}

# 4. Bilet fiyatı fonksiyonu (güvenli)
def get_ticket_price(departure_city, destination_city):
    if not departure_city or not destination_city:
        return "Error: both departure and destination cities are required."
    key = (departure_city.lower(), destination_city.lower())
    price = ticket_prices.get(key)
    return price if price else f"Estimated price from {departure_city.title()} to {destination_city.title()} is around $1000."

# 5. PNR üretici & rezervasyon fonksiyonu (tarih destekli)
def generate_pnr():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

def make_reservation(departure_city, destination_city, travel_date):
    pnr = generate_pnr()
    # travel_date format 'YYYY-MM-DD'
    try:
        fly_date = datetime.fromisoformat(travel_date)
        date_str = fly_date.strftime("%d %B %Y")
    except:
        date_str = travel_date
    return (
        f"Your reservation from {departure_city.title()} to {destination_city.title()} on {date_str} is confirmed.\n"
        f"Flight departs at 13:45 local time.\n"
        f"Reservation Code (PNR): {pnr}.\n"
        f"Seat selection will be available during check-in."
    )

# 6. Fonksiyon sarmalayıcıları
def get_ticket_price_fn(departure_city: str, destination_city: str) -> str:
    return get_ticket_price(departure_city, destination_city)

def make_reservation_fn(departure_city: str, destination_city: str, travel_date: str) -> str:
    return make_reservation(departure_city, destination_city, travel_date)

# 7. Fonksiyon tanımları
functions = [
    {
        "name": "get_ticket_price",
        "description": "Returns the ticket price between a departure and destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."}
            },
            "required": ["departure_city", "destination_city"]
        }
    },
    {
        "name": "make_reservation",
        "description": "Reserves a flight from departure to destination city on a given date and returns reservation details.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."},
                "travel_date": {"type": "string", "description": "Travel date in YYYY-MM-DD format."}
            },
            "required": ["departure_city", "destination_city", "travel_date"]
        }
    }
]

# 8. Sistem mesajı
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Never call any functions unless both the departure city, destination city, and travel date are clearly provided. "
    "If the user mentions only destination or departure, ask for the missing details. "
    "Ask explicitly for travel date in YYYY-MM-DD format."
)

# 9. Ana sohbet fonksiyonu
def chat(message, history):
    msgs = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # Modeli çağır ve varsa fonksiyon isteği al
    response = openai.chat.completions.create(
        model=MODEL,
        messages=msgs,
        functions=functions
    )
    msg = response.choices[0].message
    # Eğer fonksiyon çağırıldıysa
    if msg.function_call:
        fname = msg.function_call.name
        args = json.loads(msg.function_call.arguments)
        # Parametre kontrolü
        if fname == "get_ticket_price":
            result = get_ticket_price_fn(**args)
        else:
            # Önce rezervasyon için tarih parametresini al
            result = make_reservation_fn(**args)
        # Fonksiyon çağrı ve yanıtını ekle
        msgs.append({"role": "assistant", "content": None, "function_call": msg.function_call.to_dict()})
        msgs.append({"role": "function", "name": fname, "content": json.dumps(result)})
        # Son cevabı al
        response = openai.chat.completions.create(model=MODEL, messages=msgs)
        return response.choices[0].message.content
    return msg.content

# 10. Gradio arayüzünü başlat
gr.ChatInterface(fn=chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7903

To create a public link, set `share=True` in `launch()`.
